<a href="https://colab.research.google.com/github/chitraju-chaithanya/personalAIAgent/blob/main/HomeMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Personalized AI Agent


In [10]:
!pip install langchain==0.0.305
!pip install openai==0.28.1
!pip install pydantic>=1.10.12
!pip install pytest>=7.4.0
!pip install sentence-transformers>=2.2.0
!pip install transformers>=4.31.0
!pip install jupyter==1.0.0
!pip install chromadb==0.4.15
!pip install tiktoken

Provide api_key

In [11]:
import os
os.environ["OPENAI_API_KEY"] ="OPENAI_API_KEY"


Using a Large Language Model (LLM) to generate at least 10 diverse and realistic real estate listings containing facts about the real estate.

Results are then saved to Home.csv file.

In [15]:

prompt = """
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

Example:
Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
Green Oaks,"$800,000",3,2,"2,000 sqft","Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom", "2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure".

Consider above parameters and create a unique 10 listing in a csv format with Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description as columns.

Must have more than 10 unique listings in csv format is needed with unique Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description as columns.
"""

messages = [{"role": "system", "content": prompt}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

print(response.choices[0].message.content)

save_file = open("Home.csv", "w")
save_file.write(response.choices[0].message.content)
save_file.close()

Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
Green Meadows,"$750,000",4,3,"2,500 sqft","Modern 4-bedroom home with a spacious backyard and updated kitchen. Perfect for a growing family.","Green Meadows is a family-friendly neighborhood with parks, schools, and easy access to shopping centers."
Pine Ridge,"$820,000",3,2.5,"2,300 sqft","Beautiful 3-bedroom home with a cozy fireplace and stunning mountain views. Ideal for nature lovers!","Pine Ridge offers a peaceful retreat with hiking trails, wildlife sightings, and a strong sense of community."
Maple Grove,"$780,000",5,4,"3,000 sqft","Spacious 5-bedroom home with a large backyard and bonus room. Great for entertaining guests!","Maple Grove is known for its top-rated schools, tree-lined streets, and friendly neighbors."
Sunset Hills,"$795,000",3,2.5,"2,100 sqft","Charming 3-bedroom home with a wrap-around porch and updated bathrooms. A perfect blend of vintage and modern.","Sunset Hills boasts sc

Display data from Home.csv file.

In [16]:
import pandas as pd

df = pd.read_csv("./Home.csv")
print(df)


          Neighborhood     Price  Bedrooms  Bathrooms  House Size  \
0        Green Meadows  $750,000         4        3.0  2,500 sqft   
1           Pine Ridge  $820,000         3        2.5  2,300 sqft   
2          Maple Grove  $780,000         5        4.0  3,000 sqft   
3         Sunset Hills  $795,000         3        2.5  2,100 sqft   
4            Riverbend  $810,000         4        3.0  2,800 sqft   
5  Meadowbrook Heights  $775,000         3        2.0  2,150 sqft   
6      Oakwood Estates  $830,000         4        3.5  2,600 sqft   
7         Willow Creek  $790,000         3        2.5  2,250 sqft   
8          Cedar Ridge  $805,000         4        2.5  2,400 sqft   
9         Golden Hills  $785,000         3        2.0  2,050 sqft   

                                         Description  \
0  Modern 4-bedroom home with a spacious backyard...   
1  Beautiful 3-bedroom home with a cozy fireplace...   
2  Spacious 5-bedroom home with a large backyard ...   
3  Charming 3-be

Load data from csv file and persist in chroma vector database.

In [17]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# Configuration
CHROMA_PATH = "chroma"
CHROMA_SETTINGS = {"name": "home_listings", "path": CHROMA_PATH}

#load data from csv
loader = CSVLoader(file_path='./Home.csv')
docs = loader.load()


splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

#save data to chroma vector db.
db = Chroma.from_documents(split_docs, embeddings)




Logic for Searching and Augmenting Listing Descriptions using LLM.

In [31]:

from langchain.chat_models import ChatOpenAI

query = "I'm looking for a home with 4 bedrooms"
print("-------------------")
print("Query: ", query)
print("-------------------")


print("Results from similarity search")
print("-------------------")

#Perform similarity search for top 3 results
#It also considers for personalized augmentation based on user preference.
context = db.similarity_search_with_relevance_scores(query, k=3, augmentation_kwargs={"personalize": True})

#context from similarity search
for doc, score in context:
  print(f"Document: {doc.page_content}")
  print(f"Score: {score}")
  print("-------------------")


prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )

model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
summary = model.predict(prompt.format(query=query, context=context))

#Results from user preference
print("*****************************")
print("Personalized results from user preference")
print("*****************************")
print(summary)



-------------------
Query:  I'm looking for a home with 4 bedrooms
-------------------
Results from similarity search
-------------------
Document: Neighborhood: Meadowbrook Heights
Price: $795,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,400 sqft
Description: Spacious 4-bedroom home with a large backyard and mountain views
Neighborhood Description: Meadowbrook Heights is a peaceful community with hiking trails and a community park.
Score: 0.7384085040465864
-------------------
Document: Neighborhood: Green Meadows
Price: $750,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft
Description: Modern 4-bedroom home with a spacious backyard and updated kitchen. Perfect for a growing family.
Neighborhood Description: Green Meadows is a family-friendly neighborhood with parks, schools, and easy access to shopping centers.
Score: 0.7365810588420025
-------------------
Document: Neighborhood: Green Meadows
Price: $750,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft
Description: Modern 4-bedr

References:

1. https://github.com/
2. https://chat.openai.com/
3. https://udacity.com/